In [1]:
import joblib
from __future__ import division
import argparse 
import numpy as np
import os
import GPy
import matplotlib.pyplot as plt
from fipy import *
from scipy.interpolate import griddata
from pdb import set_trace as keyboard
import time
import random 

In [2]:
kernels = {'rbf':GPy.kern.RBF, 'exp':GPy.kern.Exponential, 
           'mat32':GPy.kern.Matern32, 'mat52':GPy.kern.Matern52}

num_samples = 1000
nx = 127
ellx = 0.1
variance = 1.0
k_ = "exp"
assert k_ in kernels.keys()
kern = kernels[k_]

os.environ['PYTHONHASHSEED'] = '0'

save_path = f"lx={ellx}"
if not os.path.exists(save_path):
    os.mkdir(save_path)

seed = 2022
# Setting the seed for numpy-generated random numbers
np.random.seed(seed=seed)

# Setting the seed for python random numbers
random.seed(seed)

#define a mean function
def mean(x):
    """
    Mean of the conductivity field. 

    m(x) = 0. 
    """
    n = x.shape[0]
    return np.zeros((n, 1))

#GPy kernel
k = kern(1, lengthscale = ellx, variance = variance)

#defining mesh to get cellcenters
Lx = 1.  # always put . after 1 
mesh = Grid1D(nx=nx, dx=Lx/nx) # with nx number of cells/cellcenters/pixels/pixelcenters
cellcenters = mesh.cellCenters.value.T  #(nx,1) matrix
cellcenters_all = np.concatenate((np.zeros((1,1)),cellcenters,np.ones((1,1))))
np.save(f"{save_path}/coordinates_nx="+str(nx)+".npy", cellcenters)


#get covariance matrix and compute its Cholesky decomposition
m = mean(cellcenters_all)
nugget = 1e-6 # This is a small number required for stability
Cov = k.K(cellcenters_all) + nugget * np.eye(cellcenters_all.shape[0])
L = np.linalg.cholesky(Cov)

#define matrices to save results 
inputs = np.zeros((num_samples, nx))
inputsall = np.zeros((num_samples, nx+2))
outputs = np.zeros((num_samples, nx))

start = time.time()
#generate samples
for i in range(num_samples):
    #display
    if (i+1)%100 == 0:
        print("Generating sample "+str(i+1))
    
    #generate a sample of the random field input
    z = np.random.randn(cellcenters_all.shape[0], 1)
    f = m + np.dot(L, z)
    sample_all = np.exp(f) # 'sample' is one image of input field: conductivity image

    # bounding input fields from below and above
    lower_bound =  np.exp(-5.298317366548036) # 0.005000000000000002
    upper_bound =  np.exp(3.5) # 33.11545195869231 

    sample_all = np.where(sample_all < lower_bound, lower_bound, sample_all) 
    sample_all  = np.where(sample_all > upper_bound, upper_bound, sample_all) 
    sample =sample_all[1:-1]


    #FIPY solution
    value_left = 1
    value_right = 0
 
    # define cell and face variables
    phi = CellVariable(name='$T(x)$', mesh=mesh, value=0.)
    D = CellVariable(name='$D(x)$', mesh=mesh, value=1.0) ## coefficient in diffusion equation
    # D = FaceVariable(name='$D(x)$', mesh=mesh, value=1.0) ## coefficient in diffusion equation
    source = CellVariable(name='$f(x)$', mesh=mesh, value=1.0)
    C = CellVariable(name='$C(x)$', mesh=mesh, value=1.0)

    # apply boundary conditions
    # dirichet
    phi.constrain(value_left, mesh.facesLeft)
    phi.constrain(value_right, mesh.facesRight)
    
    # setup the diffusion problem
    eq = -DiffusionTerm(coeff=D)+ImplicitSourceTerm(coeff=C) == source

    c = 15.
    f = 10. #source

    source.setValue(f)
    C.setValue(c)

    D.setValue(sample.ravel())

    eq.solve(var=phi)
    x_fipy = mesh.cellCenters.value.T ## fipy solution (nx,1) matrix # same as cellcenters defined above
    u_fipy = phi.value[:][:, None] ## fipy solution  (nx,1) matrix

    # x_face=mesh.faceCenters.value.flatten() #cell faces location i.e.edges of the element 
    # y_face=phi.faceValue()                  #cell faces location i.e.edges of the element
  
    #save data 
    inputs[i] = sample.ravel()
    inputsall[i] = sample_all.ravel()
    outputs[i] = u_fipy.flatten()   

#end timer
finish = time.time() - start
print("Time (sec) to generate "+str(num_samples)+" MC samples : " +str(finish))

# print np.shape(inputs)
# print np.shape(outputs)
# print inputs
# print outputs

#save data
np.save(f"{save_path}/MC_samples_inputfield_"+\
            k_+"_nx="+str(nx)+\
            "_lx="+str(ellx)+\
            "_v="+str(variance)+\
            "_num_samples="+str(num_samples)+".npy", inputs)


np.save(f"{save_path}/MC_samples_inputfield_wBoundary_"+\
            k_+"_nx="+str(nx)+\
            "_lx="+str(ellx)+\
            "_v="+str(variance)+\
            "_num_samples="+str(num_samples)+".npy", inputsall)


np.save(f"{save_path}/MC_samples_u_fipy_"+\
            k_+"_nx="+str(nx)+\
            "_lx="+str(ellx)+\
            "_v="+str(variance)+\
            "_num_samples="+str(num_samples)+".npy", outputs)

# END

Generating sample 100
Generating sample 200
Generating sample 300
Generating sample 400
Generating sample 500
Generating sample 600
Generating sample 700
Generating sample 800
Generating sample 900
Generating sample 1000
Time (sec) to generate 1000 MC samples : 35.02750372886658


In [5]:
path1 = f"{save_path}/MC_samples_u_fipy_"+\
            k_+"_nx="+str(nx)+\
            "_lx="+str(ellx)+\
            "_v="+str(variance)+\
            "_num_samples="+str(num_samples)+".npy"


path3 = f"{save_path}/MC_samples_inputfield_wBoundary_"+\
            k_+"_nx="+str(nx)+\
            "_lx="+str(ellx)+\
            "_v="+str(variance)+\
            "_num_samples="+str(num_samples)+".npy"


path2 = f"{save_path}/MC_samples_inputfield_"+\
            k_+"_nx="+str(nx)+\
            "_lx="+str(ellx)+\
            "_v="+str(variance)+\
            "_num_samples="+str(num_samples)+".npy"

cellcenters_nx = np.load(f"{save_path}/coordinates_nx="+str(nx)+".npy")
cellcenters_nx = np.concatenate((np.zeros((1,1)),cellcenters_nx,np.ones((1,1))))
np.save(f"{save_path}/coordinates_nx="+str(cellcenters_nx.shape[0])+".npy", cellcenters_nx)

trainDataInputfield= np.load(path3)
trainDataU = np.load(path1)

trainDataU = np.concatenate((np.ones((trainDataU.shape[0],1)),trainDataU,np.zeros((trainDataU.shape[0],1))),axis=1)


print(trainDataInputfield.shape)
print(cellcenters_nx.shape)
print(trainDataU.shape)

(1000, 129)
(129, 1)
(1000, 129)


In [ ]:

DataU  = trainDataU
DataInputfield = trainDataInputfield
coordinates_inputfield = cellcenters_nx  
coordinates_ouput = cellcenters_nx  
print(coordinates_ouput.shape)

joblib.dump(coordinates_inputfield,f"{save_path}/coordinates_inputfield.pkl") 
joblib.dump(coordinates_ouput,f"{save_path}/coordinates_output.pkl") 


allNum = DataInputfield.shape[0]
print(allNum)
trainNumExamples,valNumExamples, = int(allNum*0.6),int(allNum*0.2)

TrainData={}
TrainData['coeff'] = DataInputfield[:trainNumExamples,:]
TrainData['sol'] = DataU[:trainNumExamples,:]

ValData={}
ValData['coeff'] = DataInputfield[trainNumExamples:(trainNumExamples+valNumExamples),:]
ValData['sol'] = DataU[trainNumExamples:(trainNumExamples+valNumExamples),:]

TestData={}
TestData['coeff'] = DataInputfield[(trainNumExamples+valNumExamples):,:]
TestData['sol'] = DataU[(trainNumExamples+valNumExamples):,:]

joblib.dump(TrainData,f"{save_path}/TrainData.pkl")
joblib.dump(ValData,f"{save_path}/ValData.pkl")
joblib.dump(TestData,f"{save_path}/TestData.pkl")

(129, 1)
1000


['lx=0.1/TestData.pkl']